In [2]:
from keras.models import Sequential
from keras.layers import InputLayer, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from tensorflow import device
from castor import director

Using TensorFlow backend.


## Model

In [3]:
NAME = "pollux"

In [4]:
def new_model(name=NAME):
    """Model definition"""
    model = Sequential(name=name)
    model.add(InputLayer((64,64,1), name="Input0"))

    model.add(Conv2D(8, (9,9), name="Conv0", input_shape=(64,64,1)))
    model.add(LeakyReLU(name="LeReLU0"))
    model.add(MaxPooling2D((2,2), padding="same", name="Max0"))
    
    model.add(Conv2D(16, (7,7), name="Conv1"))
    model.add(LeakyReLU(name="LeReLU1"))
    model.add(MaxPooling2D((2,2), padding="same", name="Max1"))
    
    model.add(Conv2D(32, (5,5), name="Conv2"))
    model.add(LeakyReLU(name="LeReLU2"))
    model.add(MaxPooling2D((2,2), padding="same", name="Max2"))
    
    model.add(Flatten(name="Flat"))
    
    model.add(Dropout(0.2, name="Dropout"))    
    
    model.add(Dense(512, activation="tanh"))
    model.add(Dense(128, activation="tanh"))
    model.add(Dense(32, activation="tanh"))
    model.add(Dense(3, activation="linear"))
    return model

In [5]:
MODEL = new_model(NAME)
MODEL.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (Conv2D)               (None, 56, 56, 8)         656       
_________________________________________________________________
LeReLU0 (LeakyReLU)          (None, 56, 56, 8)         0         
_________________________________________________________________
Max0 (MaxPooling2D)          (None, 28, 28, 8)         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 22, 22, 16)        6288      
_________________________________________________________________
LeReLU1 (LeakyReLU)          (None, 22, 22, 16)        0         
_________________________________________________________________
Max1 (MaxPooling2D)  

In [6]:
MODEL.compile(optimizer="sgd", loss="mse", metrics=['accuracy'])

In [7]:
with device('/cpu:0'):
    MODEL.fit_generator(director("train", NAME),
                        steps_per_epoch=128, epochs=1,
                        validation_data=director("tests", NAME),
                        validation_steps=48)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
128/128 [==============================] - 1s 9ms/step - loss: 1.5995 - acc: 0.5156 - val_loss: 1.5452 - val_acc: 0.3750


In [8]:
MODEL.save(NAME+".h5")